In [7]:
from pycaret.classification import *
from pycaret.classification import load_experiment

In [8]:
import pandas as pd
import numpy as np
import pickle

from tqdm import tqdm
tqdm.pandas()

import os
from pprint import pprint
import re
from collections import defaultdict
from util import *

In [9]:
# import logging,sys
# for handler in logging.root.handlers[:]:
#     logging.root.removeHandler(handler)
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - [%(levelname)s] %(filename)s: %(message)s')


In [10]:
from datetime import datetime

print("Begining Time:\n", datetime.now())

Begining Time:
 2024-02-12 23:08:10.034319


In [11]:
DATA_ADDRESS = "./data"
# os.listdir(DATA_ADDRESS)
TRAIN_DIR = os.path.join(DATA_ADDRESS,'preprocessed','train')
TEST_DIR = os.path.join(DATA_ADDRESS,'preprocessed','test')

# os.listdir(TRAIN_DIR)

## Load Data

In [12]:
data_train = load_data(
    dir_feature = TRAIN_DIR,
    file_prefix = 'source_mfcc_len5_fft2048_mels128_mfcc17_',
    dir_df_index = os.path.join(DATA_ADDRESS,'df_index_source_train.pkl'),
    n_interval=500,
    flatten=True)

100%|██████████| 4/4 [00:00<00:00, 532.58it/s]


In [13]:
data_train

,0,1,2,3,4,5,6,7,8,9,...,3663,3664,3665,3666,3667,3668,3669,3670,3671,target
0,-80.199921,-78.522278,-78.041252,-78.678085,-79.135452,-79.036118,-79.023300,-78.891022,-78.589790,-78.287941,...,0.463551,0.172204,-0.357568,-0.051849,0.275233,-0.043794,-0.308624,-0.698927,-0.431659,4967.0
1,-54.440174,-55.169106,-55.757004,-56.125408,-56.542290,-56.143124,-54.699898,-53.520699,-52.859917,-53.442547,...,-1.133153,-0.999535,-0.878778,-0.591550,-0.171469,0.112264,0.066884,0.534677,1.084320,4967.0
2,-57.445320,-55.021637,-55.360378,-59.159683,-65.186165,-54.589661,-52.513134,-55.607136,-57.488655,-59.600983,...,0.008717,-0.693536,-0.626800,0.141316,-0.014385,0.379013,0.452272,0.804542,1.672258,4967.0
3,-36.961510,-35.436325,-37.526371,-36.496399,-41.023388,-51.365696,-53.648964,-53.890499,-52.856178,-52.297760,...,0.023778,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,4967.0
4,-77.139420,-77.644806,-78.030930,-78.258194,-75.489655,-73.090729,-75.297333,-75.117676,-73.656136,-71.496193,...,0.210924,-0.147249,-0.700613,-1.409750,-1.428489,-0.651570,-0.245459,0.205517,0.488121,4967.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,-57.105099,-51.955948,-50.820438,-52.583439,-54.693214,-55.470367,-55.521866,-56.577335,-59.098118,-57.754917,...,0.023778,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,0.023779,1066.0
1548,-82.117111,-81.464203,-81.605888,-81.727684,-81.831978,-81.693909,-81.362572,-81.190079,-80.601349,-80.457680,...,-0.844476,-1.730057,-2.055181,-1.352462,-0.604014,-0.237128,-0.023271,0.053701,0.390178,1066.0
1549,-52.875813,-57.756168,-60.996700,-63.237934,-64.417656,-64.876518,-64.905571,-64.248718,-64.266571,-65.048233,...,-0.164985,-0.697857,-0.227485,-0.362820,-0.784887,-1.591975,-1.877816,-1.888824,-0.897276,1066.0
1550,-55.924938,-61.798183,-72.218048,-68.481514,-61.453598,-59.782471,-61.291439,-61.916386,-64.142639,-67.531990,...,-0.273782,-0.788201,-0.488352,-0.765481,-0.688396,-0.582176,-0.205527,-0.075578,0.387838,1066.0


## Load Model

In [14]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.WARNING)


In [15]:
# load_experiment("model/test_source/experiment_1707607588",data)

In [16]:
# logistic regression
clf = load_model("model/test_source/logesticRegression")


Transformation Pipeline and Model Successfully Loaded


In [17]:
clf

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(include=['0', '1', '2', '3', '4', '5', '6',
                                             '7', '8', '9', '10', '11', '12',
                                             '13', '14', '15', '16', '17', '18',
                                             '19', '20', '21', '22', '23', '24',
                                             '25', '26', '27', '28', '29', ...],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('trained_model',
                 LogisticRegression(max_iter=1000, random_state=42))])

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, cohen_kappa_score, matthews_corrcoef
from sklearn.preprocessing import label_binarize

In [19]:
print(clf)

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(include=['0', '1', '2', '3', '4', '5', '6',
                                             '7', '8', '9', '10', '11', '12',
                                             '13', '14', '15', '16', '17', '18',
                                             '19', '20', '21', '22', '23', '24',
                                             '25', '26', '27', '28', '29', ...],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('trained_model',
                 LogisticRegression(max_iter=1000, random_state=42))])


In [20]:


# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(
    data_train.drop('target', axis=1),  # Assuming 'target' is the column with the labels
    data_train['target'],
    test_size=0.2,  # 20% for validation
    random_state=42  # For reproducibility
)

# Fit the classifier to the training data
# clf.fit(X_train, y_train)



In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score,precision_score, f1_score, cohen_kappa_score, matthews_corrcoef
from sklearn.preprocessing import label_binarize
from sklearn.exceptions import UndefinedMetricWarning
import numpy as np
import warnings

# Suppress undefined metric warnings
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Make predictions on both the training and validation sets
y_train_pred = clf.predict(X_train)
y_valid_pred = clf.predict(X_valid)

# If your target is not binary, you need to binarize it for AUC
y_train_bin = label_binarize(y_train, classes=np.unique(y_train))
y_train_pred_bin = label_binarize(y_train_pred, classes=np.unique(y_train))
y_valid_bin = label_binarize(y_valid, classes=np.unique(y_valid))
y_valid_pred_bin = label_binarize(y_valid_pred, classes=np.unique(y_valid))

# Calculate the evaluation metrics for both sets
metrics = {
    'Accuracy': (accuracy_score(y_train, y_train_pred), accuracy_score(y_valid, y_valid_pred)),
    'AUC': (roc_auc_score(y_train_bin, y_train_pred_bin, average='macro', multi_class='ovr'),
            roc_auc_score(y_valid_bin, y_valid_pred_bin, average='macro', multi_class='ovr')),
    "Precision": (precision_score(y_train, y_train_pred, average='macro'), precision_score(y_valid, y_valid_pred, average='macro')),
    'Recall': (recall_score(y_train, y_train_pred, average='macro'), recall_score(y_valid, y_valid_pred, average='macro')),
    'F1': (f1_score(y_train, y_train_pred, average='macro'), f1_score(y_valid, y_valid_pred, average='macro')),
    'Kappa': (cohen_kappa_score(y_train, y_train_pred), cohen_kappa_score(y_valid, y_valid_pred)),
    'MCC': (matthews_corrcoef(y_train, y_train_pred), matthews_corrcoef(y_valid, y_valid_pred))
}

# Create the evaluation table
evaluation_table = {
    'Metric': ['Accuracy', 'AUC', 'Recall',"Precision", 'F1', 'Kappa', 'MCC'],
    'Train': [metrics[metric][0] for metric in metrics],
    'Valid': [metrics[metric][1] for metric in metrics]
}

# Convert to DataFrame for better display
import pandas as pd
evaluation_df = pd.DataFrame(evaluation_table)
evaluation_df

,Metric,Train,Valid
0,Accuracy,0.533441,0.572347
1,AUC,0.765298,0.778376
2,Recall,0.659789,0.519979
3,Precision,0.532940,0.521415
4,F1,0.528327,0.487390
5,Kappa,0.531025,0.569597
6,MCC,0.532424,0.571483


In [22]:
# from sklearn.model_selection import cross_validate, StratifiedKFold
# from sklearn.metrics import make_scorer

# # Define the scoring functions for metrics not directly supported by cross_validate
# scoring = {
#     'accuracy': 'accuracy',
#     'roc_auc_ovr': 'roc_auc_ovr',
#     'recall_macro': 'recall_macro',
#     'f1_macro': 'f1_macro',
#     'kappa': make_scorer(cohen_kappa_score),
#     'mcc': make_scorer(matthews_corrcoef)
# }

# # Define a stratified 5-fold cross-validation iterator
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # Perform cross-validation
# cv_results = cross_validate(clf, data_train.drop('target', axis=1), data_train['target'],
#                             cv=cv, scoring=scoring, return_train_score=True)

# # Collect the cross-validation results into a DataFrame for better display
# evaluation_df = pd.DataFrame(cv_results)
# evaluation_df.index += 1  # Start index at 1 for fold numbers
# evaluation_df.index.name = 'Fold'

# # Calculate the mean and standard deviation for each metric
# evaluation_summary = pd.DataFrame({
#     'Train Mean': evaluation_df.filter(regex=r'^train_').mean(),
#     'Train Std': evaluation_df.filter(regex=r'^train_').std(),
#     'Test Mean': evaluation_df.filter(regex=r'^test_').mean(),
#     'Test Std': evaluation_df.filter(regex=r'^test_').std()
# })

# evaluation_summary
